# Evaluator Platypus Converter
This notebook covers how to use the Platypus conveter for Energyhub or EnergyPlus evaluators.

In [ ]:
import random

import platypus
from besos import eppy_funcs as ef, optimizer, pyehub_funcs as pf
from besos.evaluator import EvaluatorEH, EvaluatorEP
from besos.parameters import (
    Parameter,
    PathSelector,
    RangeParameter,
    expand_plist,
)
from besos.problem import EHProblem, EPProblem
from matplotlib import pyplot as plt
from platypus import DTLZ2, NSGAII

# Base Platypus Example
Here is a simple example of generating a simple Platypus problem from the DTLZ2 function to be solved with the NSGAII algorithm. There are other algorithms outlined: https://platypus.readthedocs.io/en/latest/experimenter.html#comparing-algorithms-visually.

In [ ]:
test = DTLZ2()  # instantiate the optimization algorithm
algorithm = NSGAII(test)  # optimize the problem using 10,000 function evaluations
algorithm.run(10000)  # plot the results using matplotlib
plt.scatter(
    [s.objectives[0] for s in algorithm.result],
    [s.objectives[1] for s in algorithm.result],
)
plt.xlim([0, 1.1])
plt.ylim([0, 1.1])
plt.xlabel("$f_1(x)$")
plt.ylabel("$f_2(x)$")
plt.show()

## Creating an Energyhub Evaluator to convert
The Platypus converter for Evaluators works with Energyhub Evaluators so long as each parameter also has a RangeParameter value_descriptor.

In [ ]:
hub = pf.get_hub()  # load the hub model specified in config.yaml
EHparameters = [
    Parameter(
        PathSelector(["LINEAR_CAPITAL_COSTS", "Boiler"]),
        value_descriptor=RangeParameter(100.0, 200.0),
        name="Boiler",
    ),
    Parameter(
        PathSelector(["LINEAR_CAPITAL_COSTS", "CHP"]),
        value_descriptor=RangeParameter(300.0, 400.0),
        name="CHP",
    ),
]
objectives = [
    "total_cost",
    "total_carbon",
]  # the objectives are the total_carbon and total_cost variables of the Energy Hub problem
EHproblem = EHProblem(
    EHparameters, objectives
)  # we make a problem out of the parameters and objectives
evaluatorEH = EvaluatorEH(
    EHproblem, hub
)  # and an Evaluator by combining with the hub model

In [ ]:
EHproblem

#### Converting Evaluator to Platypus problem
There's a converter in the evaluator module that makes the evaluator the function of a platypus problem with problem details from the evaluator's problem which are converted in the problem module.

In [ ]:
platypus_problem = evaluatorEH.to_platypus()
EHalgorithm = NSGAII(platypus_problem)

In [ ]:
EHalgorithm.run(5)
results1 = optimizer.solutions_to_df(
    EHalgorithm.result, EHproblem, parts=["inputs", "outputs"]
)
results1

In [ ]:
best = results1.loc[results1["pareto-optimal"] == True]
best

## Creating an Energyplus Evaluator to convert

Just like the Energyhub Evaluator the Energyplus requires the parameters all have RangeParameters but this is easier with the expand_plist
that works with EnergyPlus Parameters.

It appears to require that the idf be the idf and not json. (Format returned in default get_idf() and not get_building().

In [ ]:
parameters = expand_plist(
    {
        "NonRes Fixed Assembly Window": {
            "UFactor": (0.1, 5),
            "Solar Heat Gain Coefficient": (0.01, 0.99),
        },
        "Mass NonRes Wall Insulation": {"Thickness": (0.01, 0.09)},
    }
)
objectives = ["Electricity:Facility", "Gas:Facility"]
problem = EPProblem(parameters, objectives)
idf = ef.get_idf()
evaluator = EvaluatorEP(problem, idf)

random.seed(1)
# run the first algorithm
platypus_problem = evaluator.to_platypus()
algorithm = platypus.NSGAII(problem=platypus_problem, population_size=5)
algorithm.run(5)

# run the second algorithm
generator = platypus.InjectedPopulation(algorithm.population)
alg2 = platypus.EpsMOEA(
    problem=platypus_problem, generator=generator, epsilons=3, population_size=5
)
alg2.run(5)

results = optimizer.solutions_to_df(alg2.result, problem, parts=["inputs", "outputs"])

value = results.iloc[0]["Electricity:Facility"]
results

In [ ]:
results2 = optimizer.solutions_to_df(
    algorithm.result, problem, parts=["inputs", "outputs"]
)
results2